# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [3]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [7]:
# Dataset before augmentation
data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
data1 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)

# Dataset after augmentation
data3 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
# data2 = pd.read_csv("/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)



# Check Shapes of Datasets (they must be the same)

In [9]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")
print(f"Shape of data3: {data3.shape}")

Shape of data1: (343, 25)
Shape of data2: (240, 25)
Shape of data3: (240, 25)


In [ ]:
# If the datasets do not have the same columns ---> make one dataset match the other one (order does not matter)
# Subset columns so both have the same 
# (we need to make sure we have the same categorical and numerical columns) ---> you do this after this cell
data1 = data1.iloc[:, -13:]
data1.shape

# Convert columns into Categorical (Only for Uniform and Gaussian Distributions)

In [11]:
for i in range(-9,0):
    data1.iloc[:,i] = data1.iloc[:,i].astype(int) # Integer
    data1.iloc[:,i] = data1.iloc[:,i].astype('category') # Categories
data1.dtypes 

/var/folders/vs/hd0z3zmd61j3_8jc_fymfchr0000gn/T/ipykernel_1952/784560720.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
338    0.0
339    1.0
340    0.0
341    0.0
342    1.0
Name: 16, Length: 343, dtype: category
Categories (2, float64): [0.0, 1.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data1.iloc[:,i] = data1.iloc[:,i].astype('category') # Categories
/var/folders/vs/hd0z3zmd61j3_8jc_fymfchr0000gn/T/ipykernel_1952/784560720.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
338    0.0
339    1.0
340    0.0
341    0.0
342    0.0
Name: 17, Length: 343, dtype: category
Categories (2, float64): [0.0, 1.0]' has dtype incompatible with float64, please explicitl

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object

In [ ]:
for i in range(-5,0):
    data2.iloc[:,i] = data2.iloc[:,i].astype(int) # Integer
    data2.iloc[:,i] = data2.iloc[:,i].astype('category') # Categories
data2.dtypes
#data2.head()  

# Convert columns into Categorical (Only for Stacked Distribution)

In [13]:
for column in data2.columns[-9:]:
    data2[column] = data2[column].astype('category')

for column in data3.columns[-9:]:
    data3[column] = data3[column].astype('category')

# Concat Synthetic Dataset with New generated dataset

In [21]:
# Concat (adding rows)
combined_df = np.vstack((data2, data3))

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data2.columns)

# Convert the last number of columns to categorical (to make sure they have the same)
for column in combined_data.columns[-9:]:
    combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object


# Split Dataset for combined_data

In [23]:
# Split dataset into X and y (data1)
X = combined_data.iloc[:, :-1]  # All columns except the last one
y = combined_data.iloc[:, -1]   # Only the last column


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Split Dataset for Synthetic Data

In [15]:
# data2
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Correlation between columns test (Matrix)

In [25]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)


---------------------------Correlation Matrix------------------------- 
           0         1         2         3         4         5         6   \
0   1.000000  0.117496  0.129585  0.102646  0.221888  0.118265  0.138458   
1   0.117496  1.000000  0.174447  0.215746  0.209029  0.168616  0.224183   
2   0.129585  0.174447  1.000000  0.225309  0.252376  0.170272  0.217491   
3   0.102646  0.215746  0.225309  1.000000  0.264222  0.127427  0.179586   
4   0.221888  0.209029  0.252376  0.264222  1.000000  0.342015  0.275643   
5   0.118265  0.168616  0.170272  0.127427  0.342015  1.000000  0.135395   
6   0.138458  0.224183  0.217491  0.179586  0.275643  0.135395  1.000000   
7   0.199178  0.096817  0.136016  0.075405  0.189819  0.126775  0.142344   
8   0.258177  0.149154  0.236951  0.222894  0.170911  0.188252  0.252242   
9   0.200137  0.193115  0.203322  0.184342  0.216818  0.169172  0.216939   
10  0.186764  0.170429  0.136403  0.266149  0.195199  0.195984  0.232978   
11  0.153531  0

# Save Correlation Matrix into a Dataframe for combined_data

In [27]:
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape

          0         1         2         3         4         5         6   \
0   1.000000  0.117496  0.129585  0.102646  0.221888  0.118265  0.138458   
1   0.117496  1.000000  0.174447  0.215746  0.209029  0.168616  0.224183   
2   0.129585  0.174447  1.000000  0.225309  0.252376  0.170272  0.217491   
3   0.102646  0.215746  0.225309  1.000000  0.264222  0.127427  0.179586   
4   0.221888  0.209029  0.252376  0.264222  1.000000  0.342015  0.275643   
5   0.118265  0.168616  0.170272  0.127427  0.342015  1.000000  0.135395   
6   0.138458  0.224183  0.217491  0.179586  0.275643  0.135395  1.000000   
7   0.199178  0.096817  0.136016  0.075405  0.189819  0.126775  0.142344   
8   0.258177  0.149154  0.236951  0.222894  0.170911  0.188252  0.252242   
9   0.200137  0.193115  0.203322  0.184342  0.216818  0.169172  0.216939   
10  0.186764  0.170429  0.136403  0.266149  0.195199  0.195984  0.232978   
11  0.153531  0.209403  0.242933  0.264805  0.210203  0.123206  0.313772   
12  0.141658

(16, 16)

# Save Correlation Matrix into a Dataframe for Synthetic dataset

In [19]:
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

          0         1         2         3         4         5         6   \
0   1.000000  0.153738  0.136624  0.084442  0.065896  0.182015  0.132600   
1   0.153738  1.000000  0.031240  0.146102  0.093639  0.203931  0.174417   
2   0.136624  0.031240  1.000000  0.108437  0.062278  0.083978  0.070123   
3   0.084442  0.146102  0.108437  1.000000  0.086641  0.160223  0.137962   
4   0.065896  0.093639  0.062278  0.086641  1.000000  0.190358  0.044607   
5   0.182015  0.203931  0.083978  0.160223  0.190358  1.000000  0.114466   
6   0.132600  0.174417  0.070123  0.137962  0.044607  0.114466  1.000000   
7  -0.022989  0.060006  0.086452  0.016870 -0.012884  0.135468  0.014761   
8  -0.007064  0.084242  0.181168  0.064725  0.025051  0.156622 -0.003306   
9   0.086244  0.105229  0.078262  0.158098  0.098994  0.092857  0.128217   
10  0.039575  0.137728 -0.041335 -0.034744  0.081228 -0.007523  0.080895   
11  0.065992  0.114128 -0.031285  0.034905  0.178465  0.110940  0.000335   
12  0.084564

(16, 16)

# Differences in Matrices (Frobenius Norm)

In [29]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_abs = np.linalg.norm(matrix1 - matrix2, ord='fro')   # Absolute error with Frobenius norm

frobenius_rel = frobenius_abs/np.linalg.norm(matrix1, ord='fro')    # Relative error with Frobenius norm

print(f"Frobenius norm difference (absolute error) : {frobenius_abs: .3f}")
print(f"Frobenius notrelative error : {frobenius_rel: .3f}")


Frobenius norm difference (absolute error) :  2.405
Frobenius notrelative error :  0.552


# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
